# Proposal: Bitcoin, The New Gold
Bill Henderson

CSCI-E-83

# Introduction
As Marion Laboure, senior economist and market strategist at Deutsche Bank mentions in her video about Bitcoin (Laboure 2024), mankind has always sought assets to store wealth in combination with normal financial instruments that governments use for commerce. The longest lasting and best known of these long term assets is gold. As Bitcoin (BTC) matures and loses some of its new and eccentric flavor, some desirable qualities emerge like fixed supply, ease of trade, and value. These emerging qualities have got financial minded people thinking, could BTC be the next gold?

# Project Goals
This project will endeavour to show that BTC is currently performing similarly to gold and could be seen as a replacement for gold in its reaction to market stimulus. This will be analyzed by collecting eight different BTC exchange datasets from Kaggle (Bukhari 2024) to better understand the response of BTC markets and compare those responses to the Gold market data from Novandra's Kaggle dataset (Novandra 2024). Both of these datasets show values in US Dollars (USD).

# Data
The project consist of two datasets: The BTC data (Bukhari 2024) and the gold dataset (Novandra 2024). 

The BTC dataset is eight csv files at a time resolution of one minute records. Seven of these eight csv file are each different exchanges: Binance, Bitfinex, BitMEX, Bitstamp, Coinbase, KuCoin, and OKX. The last csv file is Combined_Index, which is an averaged collection of all the datasets to show a continuous, unbroken BTC dataset, with no gaps and no erroneous values, giving a more comprehensive view of the market (Bukhari 2024). The columns in each data set are the standard financial metrics Open, High, Low, Close, and Volume (OHLCV). Number of trades and taker volumes are also included when available. 

The Gold dataset is multiple time resolution csv datasets ranging from five minutes to one month. All the datasets start on June 11, 2004 through September 20, 2024. The columns are USD based values for OHLCV metrics, much like the BTC datasets. The gold set only has the OHLCV values and doesn't have any of the volume or trade info that the BTC datasets have. The BTC datasets have varying start and stop dates for each dataset, so the maximum start date and minimum end date will be used to ensure we have as much intersecting data between the datasets as possible.

The datasets will need some clean up for use. As the data timeframes don't line up, I'll have to exclude the times that don't intersect. Also, the BTC data doesn't line up resolution wise, so the BTC data will have to be aggregated to match the resolution chosen for the gold dataset. This aggregation will be done per column by each column's specific meaning: Open will be the first Open value in the aggregation, High will be the maximum value, Low will be the minimum value, Close will be the last, and Volume will be a sum of values. Since the intent of the project is to understand long term trends and similarity, I think a single day resolution for the datasets is the best mix of detail and aggregation. 

# Exploratory Data Analysis

# Modeling Methods

# Appendix

## Bibliography

Laboure, Marion (2024). Bitcoin could potentially become the 21st century gold. [online] Available at: https://www.db.com/what-next/digital-disruption/dossier-payments/i-could-potentially-see-bitcoin-to-become-the-21st-century-gold.

Novandra Anugrah (2024). XAU/USD Gold Price Historical Data (2004-2024). [online] Kaggle.com. Available at: https://www.kaggle.com/datasets/novandraanugrah/xauusd-gold-price-historical-data-2004-2024.

Bukhari, I. (2024). Bitcoin BTC, 7 Exchanges, 1m Full Historical Data. [online] Kaggle.com. Available at: https://www.kaggle.com/datasets/imranbukhari/comprehensive-btcusd-1m-data [Accessed 19 Nov. 2024].

‌

## Code

In [ ]:
# module imports
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns